In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath('../..'))

In [ ]:
import torch
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from models.noiseprint.network import FullConvNet
from models.noiseprint.noiseprint import genNoiseprint
from models.noiseprint.utility.utilityRead import imread2f, resizeMapWithPadding
from models.noiseprint.noiseprint_blind import noiseprint_blind_post

In [ ]:
from torchvision.transforms.functional import resize

In [ ]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.compat.v1.disable_eager_execution()

In [ ]:
input_img_path = "/media/nas2/graph_sim_data/image_cam_model_splicing/test/orig_00003.png"
# input_img_path = "/home/tai/1-workdir/2-noiseprint/demo/imgs/inpainting.png"

In [ ]:
QF = 101
model_name="net"
chkpt_folder = "/media/nas2/trained_models_repository/noiseprint_tf1/%s_jpg%d/model"
chkpt_fname = chkpt_folder % (model_name, QF)

In [ ]:
img, mode = imread2f(input_img_path, channel=1)

In [ ]:
tf.compat.v1.reset_default_graph()
x_data = tf.compat.v1.placeholder(tf.float32, [1, None, None, 1], name="x_data")
net = FullConvNet(x_data, 0.9, tf.constant(False), num_levels=17)
saver = tf.compat.v1.train.Saver(net.variables_list)

with tf.compat.v1.Session() as sess:
    saver.restore(sess, chkpt_fname)

    residual = genNoiseprint(sess, net, x_data, img, QF)
    mapp, valid, range0, range1, imgsize, other = noiseprint_blind_post(residual, img)
    resized_mapp = resizeMapWithPadding(mapp, range0, range1, imgsize)
    # resized_mapp = resize(torch.tensor(mapp).unsqueeze(0), imgsize).squeeze()
    normalized_mapp = (resized_mapp - mapp.min()) / (mapp.max() - mapp.min())

In [ ]:
normalized_mapp.mean()

In [ ]:
plt.imshow(normalized_mapp > 0.20)

In [ ]:
plt.imshow(img)

In [ ]:
plt.hist(normalized_mapp.flatten(), bins=20)